In [41]:
# - *- coding: utf- 8*-
from LambdaNodeEmbedding import KerasEval
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.callbacks import History

#These are the edges between the nodes.
current = np.array( [
                    [0,0.3,1,1]
                    ], dtype='float32')

neigbours = np.array([
                     [0,1,1,1],
                     [0,0,0,1],
                     [0,1,0,0],
                     [0,0,0,0]
                     ], dtype='float32')

weights = np.array([
                   [0, 0.3,  0.1,  1,  0.3,  0.1,  1, 0.9],
                   [1, 0.1,  0.1,  1,  0.22,  0.1,  1, 0.9], 
                   [0, 0.3,  0.1,  0.9,  0.3,  0.1,  1, 0.9],
                   [0.1, 0.6,  0.3,  1,  0.7,  0.1,  1, 0.9]
                   ], dtype='float32')


current_ten = tf.keras.backend.variable (value=current, dtype='float32')
neighbours_ten = tf.keras.backend.variable (value=neigbours, dtype='float32')
weights_ten = tf.keras.backend.variable (value=weights, dtype='float32')



def Call_Function(current, neighbours, weights, hidden_dim=4):
 
    ''' 1. Calc next feature Hop with max pooling '''
    print('Weights: \n', weights.eval())
    print('Self: \n', current.eval())
    print('Neighs: \n', neighbours.eval())
    neigh_node_max_vec = K.max(neighbours, axis=1)
    
    ''' 2. Reshape for concatenation '''
    neigh_node_max_reshape =  K.reshape(neigh_node_max_vec, (1, neighbours.shape[0]))
    print('Neigh_Max_shaped: ', neigh_node_max_reshape.eval())
    
    ''' 3. Concatenate '''
    concatenation =  K.concatenate([current, neigh_node_max_reshape], axis=1)    
    print('Concantenated: ', concatenation.eval())
    
    ''' 4. Reshape weights and multiply with weights '''
    
    cc_shape = concatenation.shape
    concatenation_reshaped = K.reshape(concatenation, (cc_shape[1], cc_shape[0]))
    
    print('Shape_Concat: \n', concatenation_reshaped.shape)
    print('Shape_Weights: \n', weights.shape)
    
    multiply = K.dot(weights ,concatenation_reshaped)
    m_shape = multiply.shape
    multiply_reshaped = K.reshape(multiply, (m_shape[1], m_shape[0]))
    print('Multiply: \n', multiply_reshaped.eval())
    
    ''' 5. (IN LAYER) Perform activation '''

    return multiply_reshaped
    

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()    
    test = Call_Function(current_ten, neighbours_ten, weights_ten)
    
    
    


Weights: 
 [[0.   0.3  0.1  1.   0.3  0.1  1.   0.9 ]
 [1.   0.1  0.1  1.   0.22 0.1  1.   0.9 ]
 [0.   0.3  0.1  0.9  0.3  0.1  1.   0.9 ]
 [0.1  0.6  0.3  1.   0.7  0.1  1.   0.9 ]]
Self: 
 [[0.  0.3 1.  1. ]]
Neighs: 
 [[0. 1. 1. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]]
Neigh_Max_shaped:  [[1. 1. 1. 0.]]
Concantenated:  [[0.  0.3 1.  1.  1.  1.  1.  0. ]]
Shape_Concat: 
 (8, 1)
Shape_Weights: 
 (4, 8)
Multiply: 
 [[2.5900002 2.45      2.49      3.28     ]]
